# Group Matches

In [1]:
import pandas as pd
import numpy as np
import networkx as nx

import sys
sys.path.insert(0, '../../data/lib/')
from consts import *

## Import file

In [2]:
df_hcp = pd.read_csv('../../data/3. transformation/4_hcp_matches.csv', index_col = 0, dtype = dataframe_types)
df_hco = pd.read_csv('../../data/3. transformation/4_hco_matches.csv', index_col = 0, dtype = dataframe_types)

df_hcp_matchlist = pd.read_csv('../../data/3. transformation/4_hcp_matchlist.csv')
df_hco_matchlist = pd.read_csv('../../data/3. transformation/4_hco_matchlist.csv')

df_prioritylist = pd.read_csv('sources/prioritylist.csv')
df_prioritylist = df_prioritylist.set_index('source')

## Group

In [3]:
def group_dataset(df, match_list, output):
    df_data = df.copy()

    #Join priority
    df_data = df_data.join(other=df_prioritylist, on='source')
    df_data['priority'] = df_data['priority'].fillna(0)

    df_data['index_copy'] = df_data.index
    df_data['group'] = -1

    #Create Network
    G = nx.from_pandas_edgelist(match_list, 'source', 'target')

    #read components (single networks)
    com = nx.connected_components(G)

    #ID of first Group
    group_counter = 1
    for group in sorted(nx.connected_components(G), key=len, reverse=True):

        #Group nodes        
        for node in group:
            df_data.loc[int(node), 'group'] = group_counter
            

        #Find main address
        df_largest = df_data[df_data.group == group_counter].nlargest(1,  ['year', 'priority'])
        df_data.loc[df_largest.iloc[0].index_copy, 'main_address'] = 1

        group_counter += 1

    #Clean
    df_data.drop(axis=1, columns=['address_expand', 'location_expand', 'priority', 'index_copy'], inplace=True)

    #Save
    df_data.to_csv('../../data/3. transformation/' + output, index=False)
    
#Group datasets
group_dataset(df_hcp, df_hcp_matchlist, '5_hcp_grouped.csv')
group_dataset(df_hco, df_hco_matchlist, '5_hco_grouped.csv')

print("Done")

Done
